In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = (10, 8)
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import collections
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:
cmp_df = pd.read_csv('data/cmp.csv', sep=';')

In [3]:
cmp_df

,COMPANY_TYPE,EMPLOYEES,ID,INDUSTRY,TITLE
0,CUSTOMER,EMPLOYEES_2,13,IT,"ООО ""Интерсвязь-10"""
1,CUSTOMER,NaN,15,IT,ИП Экшиян Наталья Валерьевна
2,CUSTOMER,EMPLOYEES_1,17,3,ИП Рублев Игорь Владимирович
3,CUSTOMER,NaN,21,ENTERTAINMENT,ООО ТФ «Тургид»
4,CUSTOMER,NaN,23,3,ООО «КонУс»
5,CUSTOMER,EMPLOYEES_1,25,IT,ООО «Артикул Медиа»
6,CUSTOMER,NaN,27,OTHER,Индивидуальный предприниматель Пискунова Елен...
7,CUSTOMER,EMPLOYEES_1,29,IT,"ООО ""Берлогос"""
8,CUSTOMER,EMPLOYEES_1,11,IT,ООО «Технологии красоты»
9,CUSTOMER,EMPLOYEES_1,31,OTHER,"ООО ""Адамант"" (окна)"


In [4]:
cmp_df['COMPANY_TYPE'].value_counts()

CUSTOMER    240
1            19
PARTNER       8
5             8
2             5
3             3
OTHER         1
Name: COMPANY_TYPE, dtype: int64

In [5]:
cmp_df['CMP_TYPE_CUSTOMER'] = cmp_df['COMPANY_TYPE'].apply(lambda x: 1 if(x == 'CUSTOMER') else 0)

In [6]:
cmp_df['CMP_TYPE_PARTNER'] = cmp_df['COMPANY_TYPE'].apply(lambda x: 1 if(x == 'PARTNER') else 0)

In [7]:
cmp_df = cmp_df.drop(columns=['COMPANY_TYPE'], axis=1)

In [8]:
cmp_df['EMPLOYEES'].value_counts()

EMPLOYEES_1    223
EMPLOYEES_2      4
Name: EMPLOYEES, dtype: int64

In [9]:
cmp_df = cmp_df.drop(columns=['EMPLOYEES'], axis=1)

In [10]:
cmp_df['INDUSTRY'].value_counts()

3                223
IT                28
OTHER             18
5                  5
DELIVERY           2
1                  1
ENTERTAINMENT      1
GOVERNMENT         1
6                  1
MANUFACTURING      1
Name: INDUSTRY, dtype: int64

In [11]:
cmp_df = cmp_df.drop(columns=['INDUSTRY'], axis=1)

In [12]:
cmp_df.head()

,ID,TITLE,CMP_TYPE_CUSTOMER,CMP_TYPE_PARTNER
0,13,"ООО ""Интерсвязь-10""",1,0
1,15,ИП Экшиян Наталья Валерьевна,1,0
2,17,ИП Рублев Игорь Владимирович,1,0
3,21,ООО ТФ «Тургид»,1,0
4,23,ООО «КонУс»,1,0


________________________________________________________________________________________________________________________________

*Сделки*

________________________________________________________________________________________________________________________________

In [13]:
deal_df = pd.read_csv('data/deal.csv', sep=';')

In [14]:
deal_df

,BEGINDATE,CLOSED,CLOSEDATE,COMPANY_ID,ID,OPPORTUNITY,PROBABILITY
0,2013-10-15T03:00:00+04:00,Y,2013-10-22T03:00:00+04:00,13,53,273600.0,100.0
1,2013-10-28T03:00:00+04:00,Y,2013-11-04T03:00:00+04:00,13,101,78500.0,90.0
2,2014-04-09T03:00:00+04:00,Y,2015-11-26T03:00:00+03:00,13,663,30000.0,100.0
3,2014-06-19T03:00:00+04:00,Y,2014-06-26T03:00:00+04:00,13,995,50550.0,50.0
4,2014-06-19T03:00:00+04:00,Y,2014-06-26T03:00:00+04:00,13,997,56300.0,0.0
5,2014-08-18T03:00:00+04:00,Y,2014-08-25T03:00:00+04:00,13,1273,46550.0,0.0
6,2014-08-31T03:00:00+04:00,Y,2015-12-14T03:00:00+03:00,13,1349,82920.0,0.0
7,2013-10-15T03:00:00+04:00,Y,2013-10-22T03:00:00+04:00,13,1557,39900.0,100.0
8,2014-11-11T03:00:00+03:00,Y,2015-11-20T03:00:00+03:00,13,1689,7450.0,50.0
9,2014-12-18T03:00:00+03:00,Y,2016-10-07T03:00:00+03:00,13,1857,50000.0,0.0


In [15]:
deal_df['CLOSED'] = deal_df['CLOSED'].apply(lambda x: 1 if(x == 'Y') else 0)
deal_df['BEGINDATE'] = pd.to_datetime(deal_df['BEGINDATE'])
deal_df['CLOSEDATE'] = pd.to_datetime(deal_df['CLOSEDATE'])
deal_df['TIME_DIFF_BEGIN_CLOSE'] = (deal_df['CLOSEDATE'] - deal_df['BEGINDATE']).astype('timedelta64[h]')/24

In [16]:
deal_df['BEGINDATE'] = pd.to_datetime(deal_df['BEGINDATE'])
deal_df['CLOSEDATE'] = pd.to_datetime(deal_df['CLOSEDATE'])

In [17]:
deal_df['TIME_DIFF_BEGIN_CLOSE'] = (deal_df['CLOSEDATE'] - deal_df['BEGINDATE']).astype('timedelta64[h]')/24

In [18]:
deal_df.head()

,BEGINDATE,CLOSED,CLOSEDATE,COMPANY_ID,ID,OPPORTUNITY,PROBABILITY,TIME_DIFF_BEGIN_CLOSE
0,2013-10-14 23:00:00,1,2013-10-21 23:00:00,13,53,273600.0,100.0,7.000000
1,2013-10-27 23:00:00,1,2013-11-03 23:00:00,13,101,78500.0,90.0,7.000000
2,2014-04-08 23:00:00,1,2015-11-26 00:00:00,13,663,30000.0,100.0,596.041667
3,2014-06-18 23:00:00,1,2014-06-25 23:00:00,13,995,50550.0,50.0,7.000000
4,2014-06-18 23:00:00,1,2014-06-25 23:00:00,13,997,56300.0,0.0,7.000000


In [19]:
deal_group = deal_df.groupby(by='COMPANY_ID')

In [20]:
deal_count = pd.DataFrame(deal_group['CLOSED'].count())

In [21]:
deal_count.head()

,CLOSED
COMPANY_ID,
3,3
5,8
11,4
13,78
17,2


In [22]:
deal_date_max = deal_group['CLOSEDATE'].max()

In [23]:
deal_date_max.head()

COMPANY_ID
3    2018-08-01
5    2018-06-04
11   2018-05-22
13   2018-12-17
17   2016-05-20
Name: CLOSEDATE, dtype: datetime64[ns]

In [24]:
deal_date_min = deal_group['BEGINDATE'].min()

In [25]:
d={'YEAR':(deal_date_max - deal_date_min).astype('timedelta64[h]') / (24 * 365)}

In [26]:
deal_date_max_min_diff = pd.DataFrame(data=d)

In [27]:
deal_date_max_min_diff.head()

,YEAR
COMPANY_ID,
3,5.315183
5,4.989155
11,4.945320
13,5.175457
17,1.024658


In [28]:
deal_by_year = pd.DataFrame()

In [29]:
deal_by_year['DEAL_BY_YEAR'] = (deal_count['CLOSED'] / deal_date_max_min_diff['YEAR']).astype(np.float32)

In [30]:
deal_by_year.head()

,DEAL_BY_YEAR
COMPANY_ID,
3,0.564421
5,1.603478
11,0.808846
13,15.071134
17,1.951872


In [31]:
deal_quantile01 = deal_group['OPPORTUNITY', 'PROBABILITY', 'TIME_DIFF_BEGIN_CLOSE'].quantile(0.1).astype(np.float32)

In [32]:
deal_quantile01.head()

0.1,OPPORTUNITY,PROBABILITY,TIME_DIFF_BEGIN_CLOSE
COMPANY_ID,,,
3,1262.400024,10.0,7.2
5,0.000000,0.0,7.0
11,0.000000,0.0,7.0
13,3213.000000,0.0,11.0
17,0.000000,0.0,16.9


In [33]:
deal_quantile09 = deal_group['OPPORTUNITY', 'PROBABILITY', 'TIME_DIFF_BEGIN_CLOSE'].quantile(0.9).astype(np.float32)

In [34]:
deal_quantile09.head()

0.9,OPPORTUNITY,PROBABILITY,TIME_DIFF_BEGIN_CLOSE
COMPANY_ID,,,
3,3880.0,90.0,45.599998
5,113418.0,62.0,570.799988
11,9970.0,40.0,96.599998
13,231072.5,100.0,722.099976
17,0.0,0.0,104.099998


In [35]:
deal_mean = deal_group['OPPORTUNITY', 'PROBABILITY', 'TIME_DIFF_BEGIN_CLOSE', 'CLOSED'].mean().astype(np.float32)

In [36]:
deal_mean.head()

,OPPORTUNITY,PROBABILITY,TIME_DIFF_BEGIN_CLOSE,CLOSED
COMPANY_ID,,,,
3,2476.000000,50.000000,24.666666,0.666667
5,46211.250000,18.571428,221.875000,1.000000
11,4975.000000,16.666666,39.000000,0.750000
13,61099.527344,31.111111,301.373383,0.807692
17,0.000000,0.000000,60.500000,1.000000


In [37]:
deal_median = deal_group['OPPORTUNITY', 'TIME_DIFF_BEGIN_CLOSE'].median().astype(np.float32)

In [38]:
deal_median.head()

,OPPORTUNITY,TIME_DIFF_BEGIN_CLOSE
COMPANY_ID,,
3,2000.0,16.0
5,29500.0,102.0
11,4950.0,7.0
13,25462.5,211.0
17,0.0,60.5


In [39]:
deal_result = pd.merge(deal_quantile01, deal_quantile09, on='COMPANY_ID', suffixes=['_DEAL_Q01','_DEAL_Q09'])

In [40]:
deal_result.head()

,OPPORTUNITY_DEAL_Q01,PROBABILITY_DEAL_Q01,TIME_DIFF_BEGIN_CLOSE_DEAL_Q01,OPPORTUNITY_DEAL_Q09,PROBABILITY_DEAL_Q09,TIME_DIFF_BEGIN_CLOSE_DEAL_Q09
COMPANY_ID,,,,,,
3,1262.400024,10.0,7.2,3880.0,90.0,45.599998
5,0.000000,0.0,7.0,113418.0,62.0,570.799988
11,0.000000,0.0,7.0,9970.0,40.0,96.599998
13,3213.000000,0.0,11.0,231072.5,100.0,722.099976
17,0.000000,0.0,16.9,0.0,0.0,104.099998


In [41]:
deal_result1 = pd.merge(deal_mean, deal_median, on='COMPANY_ID', suffixes=['_DEAL_MEAN','_DEAL_MEDIAN'])

In [42]:
deal_result1.head()

,OPPORTUNITY_DEAL_MEAN,PROBABILITY,TIME_DIFF_BEGIN_CLOSE_DEAL_MEAN,CLOSED,OPPORTUNITY_DEAL_MEDIAN,TIME_DIFF_BEGIN_CLOSE_DEAL_MEDIAN
COMPANY_ID,,,,,,
3,2476.000000,50.000000,24.666666,0.666667,2000.0,16.0
5,46211.250000,18.571428,221.875000,1.000000,29500.0,102.0
11,4975.000000,16.666666,39.000000,0.750000,4950.0,7.0
13,61099.527344,31.111111,301.373383,0.807692,25462.5,211.0
17,0.000000,0.000000,60.500000,1.000000,0.0,60.5


In [43]:
deal_result = pd.merge(deal_result, deal_result1, on='COMPANY_ID')

In [44]:
deal_result = pd.merge(deal_result, deal_by_year, on='COMPANY_ID')

In [45]:
deal_result = deal_result.mask(np.isinf(deal_result))

In [46]:
deal_result.head()

,OPPORTUNITY_DEAL_Q01,PROBABILITY_DEAL_Q01,TIME_DIFF_BEGIN_CLOSE_DEAL_Q01,OPPORTUNITY_DEAL_Q09,PROBABILITY_DEAL_Q09,TIME_DIFF_BEGIN_CLOSE_DEAL_Q09,OPPORTUNITY_DEAL_MEAN,PROBABILITY,TIME_DIFF_BEGIN_CLOSE_DEAL_MEAN,CLOSED,OPPORTUNITY_DEAL_MEDIAN,TIME_DIFF_BEGIN_CLOSE_DEAL_MEDIAN,DEAL_BY_YEAR
COMPANY_ID,,,,,,,,,,,,,
3,1262.400024,10.0,7.2,3880.0,90.0,45.599998,2476.000000,50.000000,24.666666,0.666667,2000.0,16.0,0.564421
5,0.000000,0.0,7.0,113418.0,62.0,570.799988,46211.250000,18.571428,221.875000,1.000000,29500.0,102.0,1.603478
11,0.000000,0.0,7.0,9970.0,40.0,96.599998,4975.000000,16.666666,39.000000,0.750000,4950.0,7.0,0.808846
13,3213.000000,0.0,11.0,231072.5,100.0,722.099976,61099.527344,31.111111,301.373383,0.807692,25462.5,211.0,15.071134
17,0.000000,0.0,16.9,0.0,0.0,104.099998,0.000000,0.000000,60.500000,1.000000,0.0,60.5,1.951872


--------------------------------------------------------------------------------------------------------------------------------

*СЧЕТА*

--------------------------------------------------------------------------------------------------------------------------------

In [47]:
inv_df = pd.read_csv('data/inv.csv', sep=';')

In [48]:
inv_df

,DATE_BILL,DATE_PAYED,DATE_PAY_BEFORE,PAYED,PRICE,STATUS_ID,UF_COMPANY_ID
0,2013-07-18T03:00:00+04:00,2016-10-18T07:26:24+03:00,NaN,N,38525.0,P,13
1,2013-07-18T03:00:00+04:00,2013-11-05T10:59:44+04:00,NaN,Y,38525.0,P,13
2,2013-07-18T03:00:00+04:00,2013-11-05T10:58:54+04:00,NaN,Y,74550.0,P,13
3,2013-10-27T03:00:00+04:00,2013-11-05T10:52:49+04:00,NaN,Y,19000.0,P,13
4,2013-10-27T03:00:00+04:00,2013-12-13T14:21:53+04:00,NaN,Y,59500.0,P,13
5,2013-11-17T03:00:00+04:00,2013-12-25T08:25:05+04:00,NaN,Y,74550.0,P,13
6,2013-11-17T03:00:00+04:00,2013-12-03T19:34:45+04:00,NaN,Y,44175.0,P,13
7,2013-12-12T03:00:00+04:00,2014-08-05T14:23:56+04:00,NaN,Y,44175.0,P,13
8,2013-12-12T03:00:00+04:00,2013-12-25T08:24:41+04:00,NaN,Y,80275.0,P,13
9,2014-03-19T03:00:00+04:00,2014-03-25T13:45:30+04:00,NaN,Y,30400.0,P,13


In [49]:
inv_df['DATE_BILL'] = pd.to_datetime(inv_df['DATE_BILL'])
inv_df['DATE_PAYED'] = pd.to_datetime(inv_df['DATE_PAYED'])
inv_df['DATE_PAY_BEFORE'] = pd.to_datetime(inv_df['DATE_PAY_BEFORE'])

In [50]:
inv_df['TIME_DIFF_PAYED_BILL'] = (inv_df['DATE_PAYED'] - inv_df['DATE_BILL']).astype('timedelta64[h]')/24
inv_df['TIME_DIFF_PAYBEF_PAYED'] = (inv_df['DATE_PAY_BEFORE'] - inv_df['DATE_PAYED']).astype('timedelta64[h]')/24

In [51]:
inv_df['PAYED'] = inv_df['PAYED'].apply(lambda x: 1 if(x == 'Y') else 0)

In [52]:
inv_df['STATUS_ID'].value_counts()

P    1502
D     259
N      19
T       4
Name: STATUS_ID, dtype: int64

In [53]:
inv_df['STATUS_ID_P'] = inv_df['STATUS_ID'].apply(lambda x: 1 if(x == 'P') else 0)
inv_df['STATUS_ID_D'] = inv_df['STATUS_ID'].apply(lambda x: 1 if(x == 'D') else 0)
inv_df['STATUS_ID_N'] = inv_df['STATUS_ID'].apply(lambda x: 1 if(x == 'N') else 0)
inv_df['STATUS_ID_T'] = inv_df['STATUS_ID'].apply(lambda x: 1 if(x == 'T') else 0)

In [54]:
inv_df.head()

,DATE_BILL,DATE_PAYED,DATE_PAY_BEFORE,PAYED,PRICE,STATUS_ID,UF_COMPANY_ID,TIME_DIFF_PAYED_BILL,TIME_DIFF_PAYBEF_PAYED,STATUS_ID_P,STATUS_ID_D,STATUS_ID_N,STATUS_ID_T
0,2013-07-17 23:00:00,2016-10-18 04:26:24,NaT,0,38525.0,P,13,1188.208333,NaN,1,0,0,0
1,2013-07-17 23:00:00,2013-11-05 06:59:44,NaT,1,38525.0,P,13,110.291667,NaN,1,0,0,0
2,2013-07-17 23:00:00,2013-11-05 06:58:54,NaT,1,74550.0,P,13,110.291667,NaN,1,0,0,0
3,2013-10-26 23:00:00,2013-11-05 06:52:49,NaT,1,19000.0,P,13,9.291667,NaN,1,0,0,0
4,2013-10-26 23:00:00,2013-12-13 10:21:53,NaT,1,59500.0,P,13,47.458333,NaN,1,0,0,0


In [55]:
 inv_group = inv_df.groupby(by='UF_COMPANY_ID')

In [56]:
inv_date_max = inv_group['DATE_PAYED'].max()

In [57]:
inv_date_max.head()

UF_COMPANY_ID
3    2018-08-17 11:20:41
5    2017-03-03 12:37:48
11   2013-08-08 09:35:57
13   2018-12-04 06:49:33
15   2013-07-26 05:15:24
Name: DATE_PAYED, dtype: datetime64[ns]

In [58]:
inv_date_min = inv_group['DATE_PAYED'].min()

In [59]:
inv_date_min.head()

UF_COMPANY_ID
3    2015-05-14 10:45:49
5    2015-12-25 12:32:41
11   2013-08-08 09:35:57
13   2013-11-05 06:52:49
15   2013-07-26 05:15:24
Name: DATE_PAYED, dtype: datetime64[ns]

In [60]:
inv_month_together = pd.DataFrame()

In [61]:
inv_month_together['MONTH_TOGETHER_INV'] = (inv_date_max - inv_date_min).astype('timedelta64[h]')/(24*30)

In [62]:
inv_month_together.head()

,MONTH_TOGETHER_INV
UF_COMPANY_ID,
3,39.700000
5,14.466667
11,0.000000
13,61.831944
15,0.000000


In [63]:
inv_count = pd.DataFrame(inv_group['PAYED'].count())

In [64]:
inv_count.head()

,PAYED
UF_COMPANY_ID,
3,3
5,3
11,1
13,146
15,1


In [65]:
inv_by_year = pd.DataFrame(data={'DEAL_BY_YEAR': (inv_count['PAYED'] / inv_month_together['MONTH_TOGETHER_INV'])*12}).astype(np.float32)

In [66]:
inv_by_year.head()

,DEAL_BY_YEAR
UF_COMPANY_ID,
3,0.906801
5,2.488479
11,inf
13,28.334867
15,inf


In [67]:
inv_quantile01 = inv_group['PRICE', 'TIME_DIFF_PAYED_BILL', 'TIME_DIFF_PAYBEF_PAYED'].quantile(0.1).astype(np.float32)

In [68]:
inv_quantile01.head()

0.1,PRICE,TIME_DIFF_PAYED_BILL,TIME_DIFF_PAYBEF_PAYED
UF_COMPANY_ID,,,
3,1350.400024,2.433333,-0.687500
5,11600.000000,42.900002,6.458333
11,4750.000000,2.416667,NaN
13,4443.750000,2.458333,-104.458336
15,2850.000000,5.250000,NaN


In [69]:
inv_quantile09 = inv_group['PRICE', 'TIME_DIFF_PAYED_BILL', 'TIME_DIFF_PAYBEF_PAYED'].quantile(0.9).astype(np.float32)

In [70]:
inv_quantile09.head()

0.9,PRICE,TIME_DIFF_PAYED_BILL,TIME_DIFF_PAYBEF_PAYED
UF_COMPANY_ID,,,
3,3880.0,36.000000,4.812500
5,25520.0,70.099998,6.458333
11,4750.0,2.416667,NaN
13,86137.5,119.125000,1.791667
15,2850.0,5.250000,NaN


In [71]:
inv_mean = inv_group['PRICE', 'TIME_DIFF_PAYED_BILL','TIME_DIFF_PAYBEF_PAYED', 'PAYED',
                       'STATUS_ID_P', 'STATUS_ID_D', 'STATUS_ID_N', 'STATUS_ID_T'].mean().astype(np.float32)

In [72]:
inv_mean.head()

,PRICE,TIME_DIFF_PAYED_BILL,TIME_DIFF_PAYBEF_PAYED,PAYED,STATUS_ID_P,STATUS_ID_D,STATUS_ID_N,STATUS_ID_T
UF_COMPANY_ID,,,,,,,,
3,2512.666748,17.736111,2.062500,1.000000,1.000000,0.000000,0.000000,0.0
5,17400.000000,56.500000,6.458333,0.666667,0.666667,0.333333,0.000000,0.0
11,4750.000000,2.416667,NaN,1.000000,1.000000,0.000000,0.000000,0.0
13,33397.183594,56.428436,-29.355207,0.876712,0.897260,0.075342,0.027397,0.0
15,2850.000000,5.250000,NaN,1.000000,1.000000,0.000000,0.000000,0.0


In [73]:
inv_median = inv_group['PRICE', 'TIME_DIFF_PAYED_BILL','TIME_DIFF_PAYBEF_PAYED'].median().astype(np.float32)

In [74]:
inv_median.head()

,PRICE,TIME_DIFF_PAYED_BILL,TIME_DIFF_PAYBEF_PAYED
UF_COMPANY_ID,,,
3,2000.0,10.333333,2.062500
5,11600.0,56.500000,6.458333
11,4750.0,2.416667,NaN
13,19950.0,21.208334,-2.270833
15,2850.0,5.250000,NaN


In [75]:
inv_result = pd.merge(inv_quantile01, inv_quantile09, on='UF_COMPANY_ID', suffixes=['_INV_Q01','_INV_Q09'])

In [76]:
inv_result.head()

,PRICE_INV_Q01,TIME_DIFF_PAYED_BILL_INV_Q01,TIME_DIFF_PAYBEF_PAYED_INV_Q01,PRICE_INV_Q09,TIME_DIFF_PAYED_BILL_INV_Q09,TIME_DIFF_PAYBEF_PAYED_INV_Q09
UF_COMPANY_ID,,,,,,
3,1350.400024,2.433333,-0.687500,3880.0,36.000000,4.812500
5,11600.000000,42.900002,6.458333,25520.0,70.099998,6.458333
11,4750.000000,2.416667,NaN,4750.0,2.416667,NaN
13,4443.750000,2.458333,-104.458336,86137.5,119.125000,1.791667
15,2850.000000,5.250000,NaN,2850.0,5.250000,NaN


In [77]:
inv_result1 = pd.merge(inv_mean, inv_median, on='UF_COMPANY_ID', suffixes=['_INV_MEAN','_INV_MEDIAN'])

In [78]:
inv_result1.head()

,PRICE_INV_MEAN,TIME_DIFF_PAYED_BILL_INV_MEAN,TIME_DIFF_PAYBEF_PAYED_INV_MEAN,PAYED,STATUS_ID_P,STATUS_ID_D,STATUS_ID_N,STATUS_ID_T,PRICE_INV_MEDIAN,TIME_DIFF_PAYED_BILL_INV_MEDIAN,TIME_DIFF_PAYBEF_PAYED_INV_MEDIAN
UF_COMPANY_ID,,,,,,,,,,,
3,2512.666748,17.736111,2.062500,1.000000,1.000000,0.000000,0.000000,0.0,2000.0,10.333333,2.062500
5,17400.000000,56.500000,6.458333,0.666667,0.666667,0.333333,0.000000,0.0,11600.0,56.500000,6.458333
11,4750.000000,2.416667,NaN,1.000000,1.000000,0.000000,0.000000,0.0,4750.0,2.416667,NaN
13,33397.183594,56.428436,-29.355207,0.876712,0.897260,0.075342,0.027397,0.0,19950.0,21.208334,-2.270833
15,2850.000000,5.250000,NaN,1.000000,1.000000,0.000000,0.000000,0.0,2850.0,5.250000,NaN


In [79]:
inv_result = pd.merge(inv_result, inv_result1, on='UF_COMPANY_ID')

In [80]:
inv_result = pd.merge(inv_result, inv_month_together, on='UF_COMPANY_ID')

In [81]:
inv_result = pd.merge(inv_result, inv_by_year, on='UF_COMPANY_ID')

In [82]:
inv_result = inv_result.mask(np.isinf(inv_result))

In [83]:
inv_result.head()

,PRICE_INV_Q01,TIME_DIFF_PAYED_BILL_INV_Q01,TIME_DIFF_PAYBEF_PAYED_INV_Q01,PRICE_INV_Q09,TIME_DIFF_PAYED_BILL_INV_Q09,TIME_DIFF_PAYBEF_PAYED_INV_Q09,PRICE_INV_MEAN,TIME_DIFF_PAYED_BILL_INV_MEAN,TIME_DIFF_PAYBEF_PAYED_INV_MEAN,PAYED,STATUS_ID_P,STATUS_ID_D,STATUS_ID_N,STATUS_ID_T,PRICE_INV_MEDIAN,TIME_DIFF_PAYED_BILL_INV_MEDIAN,TIME_DIFF_PAYBEF_PAYED_INV_MEDIAN,MONTH_TOGETHER_INV,DEAL_BY_YEAR
UF_COMPANY_ID,,,,,,,,,,,,,,,,,,,
3,1350.400024,2.433333,-0.687500,3880.0,36.000000,4.812500,2512.666748,17.736111,2.062500,1.000000,1.000000,0.000000,0.000000,0.0,2000.0,10.333333,2.062500,39.700000,0.906801
5,11600.000000,42.900002,6.458333,25520.0,70.099998,6.458333,17400.000000,56.500000,6.458333,0.666667,0.666667,0.333333,0.000000,0.0,11600.0,56.500000,6.458333,14.466667,2.488479
11,4750.000000,2.416667,NaN,4750.0,2.416667,NaN,4750.000000,2.416667,NaN,1.000000,1.000000,0.000000,0.000000,0.0,4750.0,2.416667,NaN,0.000000,NaN
13,4443.750000,2.458333,-104.458336,86137.5,119.125000,1.791667,33397.183594,56.428436,-29.355207,0.876712,0.897260,0.075342,0.027397,0.0,19950.0,21.208334,-2.270833,61.831944,28.334867
15,2850.000000,5.250000,NaN,2850.0,5.250000,NaN,2850.000000,5.250000,NaN,1.000000,1.000000,0.000000,0.000000,0.0,2850.0,5.250000,NaN,0.000000,NaN


--------------------------------------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------------------------------------------

In [84]:
quote_df = pd.read_csv('data/quote.csv', sep=';')

In [85]:
quote_df

,CLOSED,CLOSEDATE,COMPANY_ID,DATE_CREATE,DEAL_ID,ID,OPPORTUNITY,STATUS_ID
0,Y,2015-01-29T03:00:00+03:00,13,2015-01-23T13:28:34+03:00,1955.0,3,11550.0,APPROVED
1,Y,2015-02-10T03:00:00+03:00,13,2015-02-04T10:45:11+03:00,1985.0,9,0.0,APPROVED
2,Y,2015-03-22T03:00:00+03:00,13,2015-03-16T08:14:51+03:00,2155.0,13,29000.0,DECLAINED
3,Y,2015-03-30T03:00:00+03:00,13,2015-03-24T12:45:20+03:00,2195.0,17,63000.0,DECLAINED
4,Y,2015-08-17T03:00:00+03:00,13,2015-08-10T07:38:32+03:00,3861.0,21,13650.0,APPROVED
5,Y,2015-08-17T03:00:00+03:00,13,2015-08-11T13:08:56+03:00,3755.0,23,212600.0,APPROVED
6,Y,2015-08-27T03:00:00+03:00,13,2015-08-21T08:04:15+03:00,4634.0,24,0.0,APPROVED
7,Y,2015-09-09T03:00:00+03:00,13,2015-09-03T09:13:27+03:00,4792.0,26,5250.0,APPROVED
8,Y,2015-09-16T03:00:00+03:00,13,2015-09-10T11:09:17+03:00,1349.0,30,66150.0,APPROVED
9,Y,2015-09-16T03:00:00+03:00,13,2015-09-10T11:50:03+03:00,3341.0,32,4200.0,APPROVED


In [86]:
quote_df['CLOSEDATE'] = pd.to_datetime(quote_df['CLOSEDATE'])
quote_df['DATE_CREATE'] = pd.to_datetime(quote_df['DATE_CREATE'])

In [87]:
quote_df['TIME_DIFF_CREATE_CLOSE'] = (quote_df['CLOSEDATE'] - quote_df['DATE_CREATE']).astype('timedelta64[h]')/24

In [88]:
quote_df['CLOSED'] = quote_df['CLOSED'].apply(lambda x: 1 if(x == 'Y') else 0)

In [89]:
quote_df['STATUS_ID'].value_counts()

DECLAINED     18
APPROVED      17
DRAFT          6
UNANSWERED     2
RECEIVED       2
SENT           1
Name: STATUS_ID, dtype: int64

In [90]:
quote_df['STATUS_ID_DEC'] = quote_df['STATUS_ID'].apply(lambda x: 1 if(x == 'DECLAINED') else 0)
quote_df['STATUS_ID_APP'] = quote_df['STATUS_ID'].apply(lambda x: 1 if(x == 'APPROVED') else 0)
quote_df['STATUS_ID_DRA'] = quote_df['STATUS_ID'].apply(lambda x: 1 if(x == 'DRAFT') else 0)
quote_df['STATUS_ID_UNA'] = quote_df['STATUS_ID'].apply(lambda x: 1 if(x == 'UNANSWERED') else 0)
quote_df['STATUS_ID_REC'] = quote_df['STATUS_ID'].apply(lambda x: 1 if(x == 'RECEIVED') else 0)

In [91]:
quote_df.head()

,CLOSED,CLOSEDATE,COMPANY_ID,DATE_CREATE,DEAL_ID,ID,OPPORTUNITY,STATUS_ID,TIME_DIFF_CREATE_CLOSE,STATUS_ID_DEC,STATUS_ID_APP,STATUS_ID_DRA,STATUS_ID_UNA,STATUS_ID_REC
0,1,2015-01-29,13,2015-01-23 10:28:34,1955.0,3,11550.0,APPROVED,5.541667,0,1,0,0,0
1,1,2015-02-10,13,2015-02-04 07:45:11,1985.0,9,0.0,APPROVED,5.666667,0,1,0,0,0
2,1,2015-03-22,13,2015-03-16 05:14:51,2155.0,13,29000.0,DECLAINED,5.750000,1,0,0,0,0
3,1,2015-03-30,13,2015-03-24 09:45:20,2195.0,17,63000.0,DECLAINED,5.583333,1,0,0,0,0
4,1,2015-08-17,13,2015-08-10 04:38:32,3861.0,21,13650.0,APPROVED,6.791667,0,1,0,0,0


In [92]:
quote_group = quote_df.groupby(by='COMPANY_ID')

In [93]:
qoute_quantile01 = quote_group['OPPORTUNITY', 'TIME_DIFF_CREATE_CLOSE'].quantile(0.1).astype(np.float32)

In [94]:
qoute_quantile01.head()

0.1,OPPORTUNITY,TIME_DIFF_CREATE_CLOSE
COMPANY_ID,,
13,0.0,2.600000
25,135500.0,5.750000
197,311200.0,-0.458333
775,11745.0,5.458333
29659,30670.0,-0.458333


In [95]:
qoute_quantile09 = quote_group['OPPORTUNITY', 'TIME_DIFF_CREATE_CLOSE'].quantile(0.9).astype(np.float32)

In [96]:
qoute_quantile09.head()

0.9,OPPORTUNITY,TIME_DIFF_CREATE_CLOSE
COMPANY_ID,,
13,64260.0,6.250000
25,135500.0,5.750000
197,311200.0,-0.458333
775,46585.0,28.670834
29659,276030.0,-0.458333


In [97]:
qoute_mean = quote_group['CLOSED', 'OPPORTUNITY', 'TIME_DIFF_CREATE_CLOSE', 'STATUS_ID_DEC', 'STATUS_ID_APP',
                       'STATUS_ID_DRA', 'STATUS_ID_UNA', 'STATUS_ID_REC'].mean().astype(np.float32)

In [98]:
qoute_mean.head()

,CLOSED,OPPORTUNITY,TIME_DIFF_CREATE_CLOSE,STATUS_ID_DEC,STATUS_ID_APP,STATUS_ID_DRA,STATUS_ID_UNA,STATUS_ID_REC
COMPANY_ID,,,,,,,,
13,0.823529,32772.058594,5.169445,0.176471,0.647059,0.117647,0.0,0.058824
25,1.000000,135500.000000,5.750000,0.000000,1.000000,0.000000,0.0,0.000000
197,1.000000,311200.000000,-0.458333,0.000000,1.000000,0.000000,0.0,0.000000
775,1.000000,29325.000000,13.760417,0.750000,0.250000,0.000000,0.0,0.000000
29659,0.500000,153350.000000,-0.458333,0.500000,0.000000,0.000000,0.0,0.500000


In [99]:
qoute_median = quote_group['OPPORTUNITY', 'TIME_DIFF_CREATE_CLOSE'].median().astype(np.float32)

In [100]:
qoute_median.head()

,OPPORTUNITY,TIME_DIFF_CREATE_CLOSE
COMPANY_ID,,
13,16275.0,5.625000
25,135500.0,5.750000
197,311200.0,-0.458333
775,29725.0,5.500000
29659,153350.0,-0.458333


In [101]:
quo_result = pd.merge(qoute_quantile01, qoute_quantile09, on='COMPANY_ID', suffixes=['_QUO_Q01','_QUO_Q09'])

In [102]:
quo_result.head()

,OPPORTUNITY_QUO_Q01,TIME_DIFF_CREATE_CLOSE_QUO_Q01,OPPORTUNITY_QUO_Q09,TIME_DIFF_CREATE_CLOSE_QUO_Q09
COMPANY_ID,,,,
13,0.0,2.600000,64260.0,6.250000
25,135500.0,5.750000,135500.0,5.750000
197,311200.0,-0.458333,311200.0,-0.458333
775,11745.0,5.458333,46585.0,28.670834
29659,30670.0,-0.458333,276030.0,-0.458333


In [103]:
quo_result1 = pd.merge(qoute_mean, qoute_median, on='COMPANY_ID', suffixes=['_QUO_MEAN','_QUO_MEDIAN'])

In [104]:
quo_result1.head()

,CLOSED,OPPORTUNITY_QUO_MEAN,TIME_DIFF_CREATE_CLOSE_QUO_MEAN,STATUS_ID_DEC,STATUS_ID_APP,STATUS_ID_DRA,STATUS_ID_UNA,STATUS_ID_REC,OPPORTUNITY_QUO_MEDIAN,TIME_DIFF_CREATE_CLOSE_QUO_MEDIAN
COMPANY_ID,,,,,,,,,,
13,0.823529,32772.058594,5.169445,0.176471,0.647059,0.117647,0.0,0.058824,16275.0,5.625000
25,1.000000,135500.000000,5.750000,0.000000,1.000000,0.000000,0.0,0.000000,135500.0,5.750000
197,1.000000,311200.000000,-0.458333,0.000000,1.000000,0.000000,0.0,0.000000,311200.0,-0.458333
775,1.000000,29325.000000,13.760417,0.750000,0.250000,0.000000,0.0,0.000000,29725.0,5.500000
29659,0.500000,153350.000000,-0.458333,0.500000,0.000000,0.000000,0.0,0.500000,153350.0,-0.458333


In [105]:
quo_result = pd.merge(quo_result, quo_result1, on='COMPANY_ID')

In [106]:
quo_result = quo_result.mask(np.isinf(quo_result))

In [107]:
quo_result.head()

,OPPORTUNITY_QUO_Q01,TIME_DIFF_CREATE_CLOSE_QUO_Q01,OPPORTUNITY_QUO_Q09,TIME_DIFF_CREATE_CLOSE_QUO_Q09,CLOSED,OPPORTUNITY_QUO_MEAN,TIME_DIFF_CREATE_CLOSE_QUO_MEAN,STATUS_ID_DEC,STATUS_ID_APP,STATUS_ID_DRA,STATUS_ID_UNA,STATUS_ID_REC,OPPORTUNITY_QUO_MEDIAN,TIME_DIFF_CREATE_CLOSE_QUO_MEDIAN
COMPANY_ID,,,,,,,,,,,,,,
13,0.0,2.600000,64260.0,6.250000,0.823529,32772.058594,5.169445,0.176471,0.647059,0.117647,0.0,0.058824,16275.0,5.625000
25,135500.0,5.750000,135500.0,5.750000,1.000000,135500.000000,5.750000,0.000000,1.000000,0.000000,0.0,0.000000,135500.0,5.750000
197,311200.0,-0.458333,311200.0,-0.458333,1.000000,311200.000000,-0.458333,0.000000,1.000000,0.000000,0.0,0.000000,311200.0,-0.458333
775,11745.0,5.458333,46585.0,28.670834,1.000000,29325.000000,13.760417,0.750000,0.250000,0.000000,0.0,0.000000,29725.0,5.500000
29659,30670.0,-0.458333,276030.0,-0.458333,0.500000,153350.000000,-0.458333,0.500000,0.000000,0.000000,0.0,0.500000,153350.0,-0.458333


--------------------------------------------------------------------------------------------------------------------------------

In [108]:
result = pd.merge(cmp_df, deal_result, left_on='ID', right_on='COMPANY_ID', suffixes=('CMP', 'DEALS'), how='outer')

In [109]:
result = pd.merge(result, inv_result, left_on='ID', right_on='UF_COMPANY_ID', suffixes=('', 'INV'), how='outer')

In [110]:
result = pd.merge(result, quo_result, left_on='ID', right_on='COMPANY_ID', suffixes=('', 'QUO'), how='outer')

In [111]:
result['target'] = result['ID'].apply(lambda x: 1 if(x in [45033, 33262, 45960, 29695, 45282,
                                                          30298, 45322, 45060, 29665,
                                                          31222, 44682, 36678, 29671, 13, 31220,
                                                          31188, 29673, 29747,
                                                          31262, 43182, 29661, 45024, 29637,
                                                          45136, 31238, 45326, 45974, 45100, 45300]) else 0)

In [112]:
result = result.set_index('ID')

In [113]:
result = result.fillna(0)

In [114]:
result

,TITLE,CMP_TYPE_CUSTOMER,CMP_TYPE_PARTNER,OPPORTUNITY_DEAL_Q01,PROBABILITY_DEAL_Q01,TIME_DIFF_BEGIN_CLOSE_DEAL_Q01,OPPORTUNITY_DEAL_Q09,PROBABILITY_DEAL_Q09,TIME_DIFF_BEGIN_CLOSE_DEAL_Q09,OPPORTUNITY_DEAL_MEAN,...,OPPORTUNITY_QUO_MEAN,TIME_DIFF_CREATE_CLOSE_QUO_MEAN,STATUS_ID_DEC,STATUS_ID_APP,STATUS_ID_DRA,STATUS_ID_UNA,STATUS_ID_REC,OPPORTUNITY_QUO_MEDIAN,TIME_DIFF_CREATE_CLOSE_QUO_MEDIAN,target
ID,,,,,,,,,,,,,,,,,,,,,
13,"ООО ""Интерсвязь-10""",1,0,3213.000000,0.0,11.000000,231072.500000,100.0,722.099976,61099.527344,...,32772.058594,5.169445,0.176471,0.647059,0.117647,0.0,0.058824,16275.0,5.625,1
15,ИП Экшиян Наталья Валерьевна,1,0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000,0
17,ИП Рублев Игорь Владимирович,1,0,0.000000,0.0,16.900000,0.000000,0.0,104.099998,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000,0
21,ООО ТФ «Тургид»,1,0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000,0
23,ООО «КонУс»,1,0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000,0
25,ООО «Артикул Медиа»,1,0,0.000000,0.0,7.000000,195000.000000,80.0,124.000000,101481.820312,...,135500.000000,5.750000,0.000000,1.000000,0.000000,0.0,0.000000,135500.0,5.750,0
27,Индивидуальный предприниматель Пискунова Елен...,1,0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000,0
29,"ООО ""Берлогос""",1,0,10755.400391,0.0,7.000000,39482.500000,91.0,7.000000,24198.250000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000,0
11,ООО «Технологии красоты»,1,0,0.000000,0.0,7.000000,9970.000000,40.0,96.599998,4975.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000,0


In [115]:
#result.to_csv('data/dataframe.csv', sep=';', index_label='ID')

In [120]:
a = set(result.columns.values)

In [123]:
print(a)

{'TIME_DIFF_BEGIN_CLOSE_DEAL_Q01', 'OPPORTUNITY_QUO_Q01', 'OPPORTUNITY_QUO_MEDIAN', 'STATUS_ID_DRA', 'PRICE_INV_MEAN', 'OPPORTUNITY_QUO_Q09', 'MONTH_TOGETHER_INV', 'CLOSED', 'target', 'PRICE_INV_Q01', 'OPPORTUNITY_DEAL_Q01', 'STATUS_ID_P', 'STATUS_ID_APP', 'TIME_DIFF_CREATE_CLOSE_QUO_Q01', 'CMP_TYPE_CUSTOMER', 'TIME_DIFF_PAYBEF_PAYED_INV_Q01', 'STATUS_ID_REC', 'STATUS_ID_UNA', 'STATUS_ID_D', 'STATUS_ID_DEC', 'TIME_DIFF_PAYED_BILL_INV_Q01', 'CLOSEDQUO', 'TIME_DIFF_BEGIN_CLOSE_DEAL_MEDIAN', 'OPPORTUNITY_DEAL_MEAN', 'PROBABILITY_DEAL_Q01', 'OPPORTUNITY_DEAL_MEDIAN', 'DEAL_BY_YEARINV', 'TIME_DIFF_PAYBEF_PAYED_INV_Q09', 'TIME_DIFF_BEGIN_CLOSE_DEAL_Q09', 'TIME_DIFF_PAYBEF_PAYED_INV_MEAN', 'TIME_DIFF_PAYBEF_PAYED_INV_MEDIAN', 'OPPORTUNITY_QUO_MEAN', 'PROBABILITY_DEAL_Q09', 'TITLE', 'TIME_DIFF_PAYED_BILL_INV_Q09', 'PROBABILITY', 'PAYED', 'TIME_DIFF_CREATE_CLOSE_QUO_MEAN', 'TIME_DIFF_CREATE_CLOSE_QUO_MEDIAN', 'PRICE_INV_Q09', 'CMP_TYPE_PARTNER', 'STATUS_ID_N', 'DEAL_BY_YEAR', 'TIME_DIFF_BEGIN_C

In [130]:
b = set(['TITLE', 'CMP_TYPE_CUSTOMER', 'CMP_TYPE_PARTNER','OPPORTUNITY_DEAL_Q01',
 'PROBABILITY_DEAL_Q01', 'OPPORTUNITY_DEAL_Q09', 'PROBABILITY_DEAL_Q09',
 'OPPORTUNITY_DEAL_MEAN', 'PROBABILITY', 'OPPORTUNITY_DEAL_MEDIAN',
 'DEAL_BY_YEAR', 'TIME_DIFF_BEGIN_CLOSE_DEAL_Q01',
 'TIME_DIFF_BEGIN_CLOSE_DEAL_Q09', 'TIME_DIFF_BEGIN_CLOSE_DEAL_MEAN',
 'CLOSED', 'TIME_DIFF_BEGIN_CLOSE_DEAL_MEDIAN' ,'UF_COMPANY_ID',
 'PRICE_INV_Q01', 'PRICE_INV_Q09', 'PRICE_INV_MEAN' ,'PRICE_INV_MEDIAN',
 'TIME_DIFF_PAYED_BILL_INV_Q01' ,'TIME_DIFF_PAYBEF_PAYED_INV_Q01',
 'TIME_DIFF_PAYED_BILL_INV_Q09' ,'TIME_DIFF_PAYBEF_PAYED_INV_Q09',
 'TIME_DIFF_PAYED_BILL_INV_MEAN' ,'TIME_DIFF_PAYBEF_PAYED_INV_MEAN' ,'PAYED',
 'STATUS_ID_P', 'STATUS_ID_D', 'STATUS_ID_N', 'STATUS_ID_T',
 'TIME_DIFF_PAYED_BILL_INV_MEDIAN', 'TIME_DIFF_PAYBEF_PAYED_INV_MEDIAN',
 'MONTH_TOGETHER_INV', 'DEAL_BY_YEARINV', 'COMPANY_IDQUO',
 'OPPORTUNITY_QUO_Q01', 'TIME_DIFF_CREATE_CLOSE_QUO_Q01',
 'OPPORTUNITY_QUO_Q09', 'TIME_DIFF_CREATE_CLOSE_QUO_Q09', 'CLOSEDQUO',
 'OPPORTUNITY_QUO_MEAN', 'TIME_DIFF_CREATE_CLOSE_QUO_MEAN', 'STATUS_ID_DEC',
 'STATUS_ID_APP', 'STATUS_ID_DRA', 'STATUS_ID_UNA', 'STATUS_ID_REC',
 'OPPORTUNITY_QUO_MEDIAN', 'TIME_DIFF_CREATE_CLOSE_QUO_MEDIAN'])

In [131]:
print(b)

{'TIME_DIFF_BEGIN_CLOSE_DEAL_Q01', 'COMPANY_IDQUO', 'OPPORTUNITY_QUO_Q01', 'OPPORTUNITY_QUO_MEDIAN', 'STATUS_ID_DRA', 'PRICE_INV_MEAN', 'OPPORTUNITY_QUO_Q09', 'MONTH_TOGETHER_INV', 'CLOSED', 'PRICE_INV_Q01', 'OPPORTUNITY_DEAL_Q01', 'STATUS_ID_P', 'STATUS_ID_APP', 'TIME_DIFF_CREATE_CLOSE_QUO_Q01', 'CMP_TYPE_CUSTOMER', 'TIME_DIFF_PAYBEF_PAYED_INV_Q01', 'STATUS_ID_REC', 'STATUS_ID_UNA', 'STATUS_ID_D', 'STATUS_ID_DEC', 'TIME_DIFF_PAYED_BILL_INV_Q01', 'CLOSEDQUO', 'TIME_DIFF_BEGIN_CLOSE_DEAL_MEDIAN', 'OPPORTUNITY_DEAL_MEAN', 'PROBABILITY_DEAL_Q01', 'UF_COMPANY_ID', 'OPPORTUNITY_DEAL_MEDIAN', 'DEAL_BY_YEARINV', 'TIME_DIFF_PAYBEF_PAYED_INV_Q09', 'TIME_DIFF_BEGIN_CLOSE_DEAL_Q09', 'TIME_DIFF_PAYBEF_PAYED_INV_MEAN', 'TIME_DIFF_PAYBEF_PAYED_INV_MEDIAN', 'OPPORTUNITY_QUO_MEAN', 'PROBABILITY_DEAL_Q09', 'TITLE', 'TIME_DIFF_PAYED_BILL_INV_Q09', 'PROBABILITY', 'PAYED', 'TIME_DIFF_CREATE_CLOSE_QUO_MEAN', 'TIME_DIFF_CREATE_CLOSE_QUO_MEDIAN', 'PRICE_INV_Q09', 'CMP_TYPE_PARTNER', 'STATUS_ID_N', 'DEAL_BY_Y

In [132]:
b - a

{'COMPANY_IDQUO', 'UF_COMPANY_ID'}